<a href="https://colab.research.google.com/github/TomMcIver/Q-Paper/blob/main/Q_Blueprint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall -y qiskit qiskit-aer qiskit-terra qiskit-ibmq-provider
!pip install qiskit qiskit-aer --upgrade
!pip install PennyLane-Lightning


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.1/56.1 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 78.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 64.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.0/930.0 kB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
from qiskit.visualization import plot_bloch_multivector, plot_histogram
import pennylane as qml

plt.rcParams["figure.figsize"] = (8, 6)


In [5]:
import numpy as np

class Qubit:
    def __init__(self):

        self.state = np.array([1, 0], dtype=complex)

    def apply_hadamard(self):
        #apply Hadamard gate
        H = np.array([[1, 1], [1, -1]]) / np.sqrt(2)
        self.state = H @ self.state

    def measure(self):
        # measure the qubit returns 0 or 1 with a rough 50% probability
        return np.random.choice([0, 1], p=np.abs(self.state)**2)


qubit = Qubit()

print("Before Hadamard:", qubit.state)


qubit.apply_hadamard()

print("After Hadamard:", qubit.state)


results = [qubit.measure() for _ in range(10)]
print("Measurement results:", results)


Before Hadamard: [1.+0.j 0.+0.j]
After Hadamard: [0.70710678+0.j 0.70710678+0.j]
Measurement results: [0, 1, 0, 1, 1, 1, 0, 1, 1, 0]


### This code shows the initialization of a Qubit of |0⟩ and |1⟩ in superposition using the Hadamrd gate.  And how the probablityes work when measured.

In [6]:
import numpy as np

class QuantumRegister:
    def __init__(self):

        self.state = np.array([1, 0, 0, 0], dtype=complex)

    def apply_hadamard(self, qubit):
         #hadamard gate to one qubit.
        H = np.array([[1, 1], [1, -1]]) / np.sqrt(2)
        gate = np.kron(H, np.eye(2)) if qubit == 0 else np.kron(np.eye(2), H)
        self.state = gate @ self.state

    def apply_cnot(self):
        """ Apply CNOT gate to entangle qubits. """
        CNOT = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0]])
        self.state = CNOT @ self.state

    def measure(self):
        """ Measure the qubits and return 00, 01, 10, or 11. """
        return np.random.choice([0, 1, 2, 3], p=np.abs(self.state)**2)

# apply hadamard & cnot aka: entanglement
register = QuantumRegister()
register.apply_hadamard(0)
register.apply_cnot()

# bell state
bell_state = register.state
print(f"Bell State: ({bell_state[0]:.2f})|00⟩ + ({bell_state[3]:.2f})|11⟩")

# measure
results = [f"{register.measure():02b}" for _ in range(10)]
print("Measurement outcomes:", ", ".join(results))


Bell State: (0.71+0.00j)|00⟩ + (0.71+0.00j)|11⟩
Measurement outcomes: 00, 00, 11, 00, 00, 11, 00, 11, 11, 11


### This creates a register of a quantum system that uses two qubits puts one in a superposition and then entangles it with another (more specifically a bell state this is mainly sued for 2 qubits). From there we get the bell state and then measure the system collapsing the qubits into there different states.